# First step in pipeline
**Load HDBSCAN and t-SNE models and find probabilities for all sentences in a document. The output of this file will be used in the segmentation**

In [8]:
import os
import pickle
import hdbscan
import numpy as np
import pandas as pd
from openTSNE import TSNE
from bertopic import BERTopic
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

In [9]:
# load PCA class object
with open('pca_model_100.pkl', 'rb') as inp:
    PCA_model = pickle.load(inp)

In [10]:
# load t-SNE class object
with open('sentence100_tsne_data_.pkl', 'rb') as inp:
    tsne_data = pickle.load(inp)

In [11]:
# load cluster model
with open('sentence100_hdbscan_model.pkl', 'rb') as inp:
    hdbscan_model = pickle.load(inp)

In [ ]:
# loop through all of the annotated transcripts and save their topic probability distribution
directory = '../Thesis/annotated_transcripts_input'

for dirpath, _, files in os.walk(directory):
    for file in files:
        episode_id = file.split('_')[0]
        path = os.path.join(dirpath, file)
        with open(path, errors='replace') as f: 
            prediction_documents = pd.read_csv(f)
            docs = prediction_documents.transcript_subset

            # embedd
            sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
            embedded_sentences = sentence_model.encode(docs)

            # reduce dimensions with PCA and t-SNE
            PCA_data = PCA_model.transform(embedded_sentences)
            print(f"{1-sum(PCA_model.explained_variance_ratio_):.2%} of the variance has been removed by PCA for transcript {episode_id}")
            tsne_prediction = tsne_data.transform(PCA_data)

            # get probability vectors for each cluster via soft clustering
            probabilities = hdbscan.membership_vector(hdbscan_model, tsne_prediction)
            print(f'nrows: {len(probabilities)}\nncols: {len(probabilities[0])}')
            # remedy HDBSCAN problem
            inds = np.where(np.isnan(probabilities))
            probabilities[inds] = 0
            print(f"Number of nan rows: {len(set(list(inds[:][0])))} for {episode_id}\n\n")

            # save probabilities to use in downstream segmentation
            name_of_file = f'../Thesis/annotated_probabilities/topic_probability_density_vector_{episode_id}.npy'
            np.save(name_of_file, probabilities)

### Code for individual transcript predicitons

In [2]:
# load dataframe of one transcript with sentences as instances
sentences = pd.read_csv('first_podcast.csv.gz', compression='gzip')
documents = sentences.transcript_subset.to_list()

In [7]:
# embedd
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embedded_sentences = sentence_model.encode(documents)

In [8]:
# reduce dimensions with PCA and t-SNE
PCA_model = PCA(n_components = 50)
PCA_data = PCA_model.fit_transform(embedded_sentences)
print(f"{1-sum(PCA_model.explained_variance_ratio_):.2%} of the variance has been removed by PCA")

tsne_test = tsne_data.transform(PCA_data)

38.83% of the variance has been removed by PCA
===> Finding 15 nearest neighbors in existing embedding using Annoy approximate search...
   --> Time elapsed: 11.78 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 0.03 seconds
===> Running optimization with exaggeration=4.00, lr=0.10 for 0 iterations...
   --> Time elapsed: 0.00 seconds
===> Running optimization with exaggeration=1.50, lr=0.10 for 250 iterations...
Iteration   50, KL divergence 9527.4070, 50 iterations in 70.7268 sec
Iteration  100, KL divergence 9287.1780, 50 iterations in 92.7301 sec
Iteration  150, KL divergence 9191.3463, 50 iterations in 104.2941 sec
Iteration  200, KL divergence 9151.7852, 50 iterations in 87.6444 sec
Iteration  250, KL divergence 9132.1676, 50 iterations in 86.1700 sec
   --> Time elapsed: 441.57 seconds


In [9]:
# get probability vectors for each cluster via soft clustering
probabilities = hdbscan.membership_vector(hdbscan_model, tsne_test)
print(f'nrows: {len(probabilities)}')
print(f'ncols: {len(probabilities[0])}')
# remedy for HDBSCAN problem
inds = np.where(np.isnan(probabilities))
probabilities[inds] = 0
print(f"Number of nan rows: {len(set(list(inds[:][0])))}")

/Users/oskarmunckafrosenschold/Documents/Thesis/thesisenv/lib/python3.10/site-packages/hdbscan/prediction.py:581: RuntimeWarning: All-NaN slice encountered
  outlier_vec = outlier_membership_vector(


nrows: 599
ncols: 156


In [11]:
# save probabilities to use in segmentation
np.save('probabilities.npy', probabilities)